In [1]:
#!/usr/bin/venv python3.7
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import datetime as dt
import numpy as np
import pandas as pd
import sys  
sys.path.insert(0, '/Users/annhuang/code/')
from strategy.utils.calculate_tte import calculate_tte
pd.options.display.float_format = '{:,.1f}'.format
import os
import pprint
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 2000)
import matplotlib.pyplot as plt
%matplotlib inline

## Question 1